# The Sardinas&ndash;Patterson Algorithm

This notebook provides a tutorial analyzing **uniquely decodable codes** using the **Sardinas–Patterson algorithm**.  

Given a codebook associated with a code, we can:

- check whether the codebook is **uniquely decodable**
- display the sequence of **SP (suffix) sets**
- identify **ambiguous messages** when the code is not uniquely decodable, and
- enumerate all **possible decodings** of those ambiguous strings.

The included examples demonstrate both a uniquely decodable codebook and one that fails the criterion, together with the corresponding ambiguous decodings.


# Imports

In [1]:
import itertools

from IPython.display import display, Math

# Function Definitions

In this section, we define the core functions for unary coding:

- `unary_encode`
- `unary_decode`
- `unary_length`
- `unary_implied_probability`

These functions form the foundation for encoding, decoding, and analyzing unary codes throughout this demo.

## `uniquely_decodable`

Determines whether the code defined by the given codebook is uniquely decodable.

In [2]:
def uniquely_decodable(codebook, trace=False):
    codebook = set(codebook)
    
    S_cur = dict() if trace else set()
    for a, b in itertools.product(codebook, codebook):
        if b.startswith(a):
            suffix = b[len(a):]
            if suffix != "":
                if trace:
                    S_cur.setdefault(suffix, set()).add((a, b, "left"))
                else:
                    S_cur.add(suffix)

    sp_sets = [S_cur] if trace else None

    while True:
        if "" in S_cur:
            return (False, sp_sets) if trace else False

        S_next = dict() if trace else set()

        for suffix, codeword in itertools.product(S_cur, codebook):
            if suffix.startswith(codeword):
                new_suffix = suffix[len(codeword):]
                if trace:
                    S_next.setdefault(new_suffix, set()).add((codeword, suffix, "left"))
                else:
                    S_next.add(new_suffix)
            
            if codeword.startswith(suffix):
                new_suffix = codeword[len(suffix):]
                if trace:
                    S_next.setdefault(new_suffix, set()).add((codeword, suffix, "right"))
                else:
                    S_next.add(new_suffix)

        if trace:
            sp_sets.append(S_next)

        if set(S_cur) == set(S_next):
            return (True, sp_sets) if trace else True

        S_cur = S_next

## `find_ambiguous_string`

Finds all ambiguous messages implied by the Sardinas–Patterson suffix sets.

In [3]:
def uniquely_decodable(codebook, trace=False):
    codebook = set(codebook)
    
    S_cur = dict() if trace else set()
    for a, b in itertools.product(codebook, codebook):
        if b.startswith(a):
            suffix = b[len(a):]
            if suffix != "":
                if trace:
                    S_cur.setdefault(suffix, set()).add((a, b, "left"))
                else:
                    S_cur.add(suffix)

    sp_sets = [S_cur] if trace else None

    while True:
        if "" in S_cur:
            return (False, sp_sets) if trace else False

        S_next = dict() if trace else set()

        for suffix, codeword in itertools.product(S_cur, codebook):
            if suffix.startswith(codeword):
                new_suffix = suffix[len(codeword):]
                if trace:
                    S_next.setdefault(new_suffix, set()).add((codeword, suffix, "left"))
                else:
                    S_next.add(new_suffix)
            
            if codeword.startswith(suffix):
                new_suffix = codeword[len(suffix):]
                if trace:
                    S_next.setdefault(new_suffix, set()).add((codeword, suffix, "right"))
                else:
                    S_next.add(new_suffix)

        if trace:
            sp_sets.append(S_next)

        if set(S_cur) == set(S_next):
            return (True, sp_sets) if trace else True

        S_cur = S_next
        
def find_ambiguous_strings(sp_sets):
    def _find_ambiguous_strings(level, suffix):
        if level < 0:
            yield "".join(map(str,left)) + "".join(map(str,reversed(right)))
        else:
            for codeword,prev_suffix,direction in sp_sets[level][suffix]:
                assert direction in {"left", "right"}
                if direction == "left":
                    left.append(codeword)
                else:
                    right.append(codeword)
                yield from _find_ambiguous_strings(
                    level-1, prev_suffix
                )
                if direction == "left":
                    left.pop()
                else:
                    right.pop()
                
    
    left = list()
    right = list()
    return set(_find_ambiguous_strings(
        len(sp_sets)-1, ""
    ))

## `find_decodings`

Determines all possible decodings of a given message using the specified codebook.

In [4]:
def find_decodings(message, codebook):
    def _find_decodings(cur_message):
        if cur_message == "":
            yield list(decoding)
        else:
            for codeword in codebook:
                if cur_message.startswith(codeword):
                    decoding.append(codeword)
                    yield from _find_decodings(
                        cur_message[len(codeword):]
                    )
                    decoding.pop()
    
    decoding = list()
    return list(
        _find_decodings(message)
    )

## `analyze_codebook`

Analyzes a codebook by applying the Sardinas–Patterson algorithm, displaying the SP sets, and reporting example ambiguous messages and their possible decodings.

In [5]:
def analyze_codebook(codebook):
    is_ud, sp_sets = uniquely_decodable(codebook, trace=True)

    print("Codebook:", ", ".join(map(str, sorted(codebook))))
    print()
    print(f"Uniquely Decodable: {is_ud}")
    print()
    print("SP Sets:")
    for ii,entry in enumerate(sp_sets, 1):
        elements = [
            r"\varepsilon" if s == "" else s for s in sorted(entry)
        ]
        set_string = f"\\qquad S_{{{ii}}} = \\{{{', '.join(elements)}\\}}"
        display(Math(set_string))
        

    if not is_ud:
        for message in find_ambiguous_strings(sp_sets):
            print()
            print("Message:", message)
            print("Decodings:")

            for decoding in find_decodings(message, codebook):
                print("\t", " | ".join(map(str, decoding)))
            print()

# Examples

## Example 1

In [6]:
codebook = [
    "0",
    "01",
    "011"
]

analyze_codebook(codebook)

Codebook: 0, 01, 011

Uniquely Decodable: True

SP Sets:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Example 2

In [7]:
codebook = [
    "01",
     "10",
     "101",
     "11101"
]

analyze_codebook(codebook)

Codebook: 01, 10, 101, 11101

Uniquely Decodable: False

SP Sets:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>


Message: 1010101
Decodings:
	 10 | 10 | 101
	 10 | 101 | 01
	 101 | 01 | 01


Message: 0110101
Decodings:
	 01 | 10 | 101
	 01 | 101 | 01



## Example 3

In [8]:
codebook = [
    "01",
     "010"
]

analyze_codebook(codebook)

Codebook: 01, 010

Uniquely Decodable: True

SP Sets:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>